In [10]:
import mlflow
import os, sys

import torch
import torch.nn.functional as F

import os; os.chdir("/home/rodrigo/CISTIB/repos/CardiacCOMA/")
from config.load_config import load_yaml_config, to_dict

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image
from mlflow.tracking import MlflowClient

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

import surgeon_pytorch
from surgeon_pytorch import Inspect, get_layers

import numpy as np
from IPython import embed
sys.path.insert(0, '..')

import model.Model3D
from utils.helpers import get_coma_args, get_lightning_module, get_datamodule
from copy import deepcopy

In [2]:
TRACKING_URI = "file:///home/rodrigo/CISTIB/repos/CardiacCOMA/mlruns"
mlflow.set_tracking_uri(TRACKING_URI)

In [6]:
# default_experiment = "Synthetic data"
#default_experiment = "Synthetic data 2"
options = [exp.name for exp in mlflow.list_experiments()]

experiment_w = widgets.Select(
    options=options,
    value=options[1]
)
display(experiment_w)

A Jupyter Widget

Retrieve run data from MLflow

In [7]:
# runs_list = mlflow.search_runs(experiment_ids=[exp_id], output_format="list")
exp_id = mlflow.get_experiment_by_name(experiment_w.value).experiment_id
runs_df = mlflow.search_runs(experiment_ids=[exp_id],)
runs_df = runs_df[runs_df.status == "FINISHED"].reset_index(drop=True)
runs_df = runs_df.set_index(["experiment_id", "run_id"])

In [9]:
experiment_id, run_id = runs_df.index[runs_df["metrics.val_recon_loss"].argmin()]

run_info = runs_df.loc[experiment_id, run_id].to_dict()
chkpt_dir = os.path.join(run_info["artifact_uri"].replace("file://", ""), "restored_model_checkpoint")
chkpt_file = os.path.join(chkpt_dir, os.listdir(chkpt_dir)[0])

model_pretrained_weights = torch.load(chkpt_file, map_location=torch.device('cpu'))["state_dict"]
model_pretrained_weights

OrderedDict([('model.encoder.layers.layer_0.graph_conv.bias',
              tensor([-1.3617e-04,  5.7601e-05, -1.4957e-04,  1.0560e-04, -2.0416e-04,
                       1.1267e-04, -3.0042e-04, -3.1896e-04, -4.3803e-05, -2.1057e-04,
                      -5.8158e-04, -2.0290e-04,  6.7160e-05, -1.8889e-04, -1.8848e-04,
                      -5.1140e-04])),
             ('model.encoder.layers.layer_0.graph_conv.lins.0.weight',
              tensor([[ 0.5181,  0.1287,  0.3220],
                      [ 0.4877,  0.1242, -0.2184],
                      [ 0.3709,  0.2744,  0.2644],
                      [-0.5443, -0.2968, -0.0886],
                      [-0.3048,  0.2775, -0.5120],
                      [ 0.4287,  0.0063, -0.0840],
                      [-0.3827,  0.3065,  0.2523],
                      [-0.4196, -0.2216, -0.4194],
                      [-0.0571,  0.4785, -0.0824],
                      [ 0.2740,  0.3649, -0.0471],
                      [ 0.5046, -0.2298,  0.4941],
       

In [11]:
ref_config = load_yaml_config("config_files/config.yaml")

def overwrite_ref_config(ref_config, run_info):
    
    config = deepcopy(ref_config)
    config.network_architecture.latent_dim = int(run_info["params.latent_dim"])
    config.loss.regularization.weight = float(run_info["params.w_kl"])
    config.optimizer.parameters.lr = float(run_info["params.lr"])
    config.sample_sizes = [100, 100, 100]
    
    return config

config = overwrite_ref_config(ref_config, run_info)
to_dict(config)

{'seed': None,
 'batch_size': 32,
 'dataset': {'data_type': 'cardiac',
  'filename': 'data/cardio/LV_meshes_at_ED_35k.pkl',
  'template': 'data/cardio/faces.pkl',
  'preprocessing': {'procrustes': 'data/cardio/procrustes_transforms_35k.pkl'}},
 'sample_sizes': [100, 100, 100],
 'network_architecture': {'latent_dim': 8,
  'n_features': 3,
  'convolution': {'channels_enc': [16, 32, 64, 128],
   'channels_dec': [16, 32, 64, 128],
   'type': 'Chebyshev',
   'parameters': {'polynomial_degree': [6, 6, 6, 6]}},
  'activation_function': ['ReLU', 'ReLU', 'ReLU', 'ReLU'],
  'pooling': {'type': 'quadric',
   'parameters': {'downsampling_factors': [2, 2, 2, 2]}}},
 'loss': {'reconstruction': {'type': 'MSE', 'weight': 1},
  'regularization': {'type': 'KL', 'weight': 0.0}},
 'optimizer': {'algorithm': 'Adam',
  'parameters': {'lr': 1e-06, 'betas': [0.5, 0.99], 'weight_decay': 0.0005}},
 'mlflow': {'run_name': None,
  'experiment_name': 'Synthetic data',
  'tracking_uri': None,
  'artifact_location':

In [ ]:
client = MlflowClient()
local_dir = "/tmp/artifact_downloads"

if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# client._tracking_client.list_artifacts(
#     runs_df.run_id[2], 'restored_model_checkp# oint'
# )

In [13]:
dm = get_datamodule(config, perform_setup=False)

In [ ]:
dm.setup(stage="test")

In [ ]:
get_coma_args(config, dm)

In [ ]:
get_lightning_module()

In [ ]:
model.Model3D.ENCODER_ARGS

In [ ]:
to_dict(config)

In [ ]:
enc_config = 
dec_config = 